In [ ]:
import os
import re
import sys
import getopt
import numpy
import scipy
import dotenv
import parmed
import pytraj
import pathlib
import itertools
import subprocess
import scipy.spatial
import subprocess as sp

work_folder = '/md/tmp'
# output files
image_pdb = os.path.join(work_folder,'image.pdb')
trim_pdb = os.path.join(work_folder,'trim.pdb') # for reduce
reduce_pdb = os.path.join(work_folder,'reduce.pdb')
reduce_log = os.path.join(work_folder,'reduce.log')
image_pka = os.path.join(work_folder,'image.pka')
image_propka_input = os.path.join(work_folder,'image.propka_input')
tleap_pdb = os.path.join(work_folder,'tleap.pdb')
protein_parm7 = os.path.join(work_folder,'protein.parm7')
protein_crd = os.path.join(work_folder,'protein.crd')
protein_lib = os.path.join(work_folder,'protein.lib')
protein_pqr = os.path.join(work_folder,'protein.pqr')
protein_pdb = os.path.join(work_folder,'protein.pdb')
protein_pdbqt = os.path.join(work_folder,'protein.pdbqt')

ENV_FILE_PATH = '/md/.env'
dotenv.load_dotenv(str(ENV_FILE_PATH))
environ = os.environ

In [19]:
os.makedirs('/md/tmp', exist_ok=True)

# 1. Initial PDB Preparation and Renumbering


In [ ]:
model_pdb = '/md/output/2SRC/model.pdb'
# --------------------------------------------------------------------------------
# Extract protein from "model.pdb".
# Residue numbers are reorder at this stage.
struct = parmed.load_file(model_pdb)
struct.save(image_pdb, format='PDB', renumber=True, standard_resnames=True, overwrite=True)

# 2. Hydrogen Addition and Histidine Protonation (using Reduce)

In [ ]:
from prepare_protein import pytraj_detect_Reduce_HIX

# --------------------------------------------------------------------------------
# Run Reduce and get modificated residues.
# Not using the flipped pdb ("reduce_pdb") in the following steps
# to preserve the original orientation of residues.
command = '{} -Trim {} > {} 2> {}'.format(environ['REDUCE_EXE'], image_pdb, trim_pdb, reduce_log)
os.system(command)
command = '{} -BUILD -FLIP {} > {} 2> {}'.format(environ['REDUCE_EXE'], trim_pdb, reduce_pdb, reduce_log)
os.system(command)

traj_reduce = pytraj.load(reduce_pdb)
reduce_modifications = pytraj_detect_Reduce_HIX(traj_reduce.top)

# 3. Ionizable Residue Protonation State Determination (using Propka)

In [13]:
from prepare_protein import Propka_get_propka_Amber_protonation_state
# --------------------------------------------------------------------------------
# Run propka31 and get modificated residues
command = '{} {}'.format(environ['PROPKA31_EXE'], image_pdb)
child = sp.Popen(command.split(), stdin=sp.PIPE, stdout=sp.PIPE, stderr=sp.PIPE, cwd=work_folder)
child.communicate()
#     os.system(command)

propka_modifications = Propka_get_propka_Amber_protonation_state(image_pka)

# 4. Disulfide Bond Detection

In [14]:
from prepare_protein import pytraj_detect_disulfide_bond
# --------------------------------------------------------------------------------
# detect disulfide bonds and get modificated residues
traj_image = pytraj.load(image_pdb)
disulfide_modifications, disulfide_bonds = pytraj_detect_disulfide_bond(traj_image.top, traj_image.xyz[0])

# 5. Generating TLeap-ready PDB

In [15]:
from prepare_protein import ParmEdPDBResnameModifier

# --------------------------------------------------------------------------------
# Generate modified pdb, "tleap.pdb", ready for tleap
struct_image = parmed.load_file(image_pdb)

M = ParmEdPDBResnameModifier(struct_image)
M.add_modifications(reduce_modifications)
M.add_modifications(propka_modifications)
M.add_modifications(disulfide_modifications)
#print(M.modifications)

struct_tleap = M.apply_modifications()['!@H=']
struct_tleap.save(tleap_pdb, format='PDB', overwrite=True)

# 6. Topology and Coordinate Generation (using TLeap)

In [ ]:
from prepare_protein import Tleap_bond_command, Program

# --------------------------------------------------------------------------------
# Generate "protein.pdb" and "protein.pqr" using tleap and ambpdb

# using tleap to create parm7 and crd file
tleap_script = '\n'.join([
    'set default PBradii mbondi2',
    'source leaprc.protein.ff14SB',
    'source leaprc.water.tip3p',
    'protein = loadpdb {}'.format(tleap_pdb),
] + Tleap_bond_command('protein', disulfide_bonds) + [
    'saveamberparm protein {} {}'.format(protein_parm7, protein_crd),
    'saveOff protein {}'.format(protein_lib),
    'quit'
])

retcode = Program(
    work_folder,
    [('tleap.in', tleap_script)],
    [(environ['TLEAP_EXE'],),
        ('-f', 'tleap.in')]
).run()
retcode.check_returncode()

-I: Adding /usr/local/amber20/dat/leap/prep to search path.
-I: Adding /usr/local/amber20/dat/leap/lib to search path.
-I: Adding /usr/local/amber20/dat/leap/parm to search path.
-I: Adding /usr/local/amber20/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
Using H(N)-modified Bondi radii
----- Source: /usr/local/amber20/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/amber20/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/amber20/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/amber20/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/amber20/dat/leap/lib/amino12.lib
Loading library: /usr/local/amber20/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/amber20/da

# 7. PQR and PDB File Generation (using ambpdb)

In [17]:
# using ambpdb to create pqr file
retcode = Program(
    work_folder,
    [],
    [(environ['AMBPDB_EXE'],),
        ('-p', protein_parm7),
        ('-pqr',),
        ('-bres',),
        ('-aatm',),
        ('< {}'.format(protein_crd),),
        ('> {}'.format(protein_pqr),)]
).run()
retcode.check_returncode()

# using ambpdb to create pdb file
retcode = Program(
    work_folder,
    [],
    [(environ['AMBPDB_EXE'],),
        ('-p', protein_parm7),
        ('-bres',),
        ('-aatm',),
        ('< {}'.format(protein_crd),),
        ('> {}'.format(protein_pdb),)]
).run()
retcode.check_returncode()

# 8. PDBQT File Generation (using AutoDockTools)

In [18]:
# --------------------------------------------------------------------------------
# using autodocktools to create pqbqt file (NO TEST)
retcode = Program(
    work_folder,
    [],
    [(environ['PYTHONSH_EXE'],),
        (os.path.join(environ['UTILITIES24'], 'prepare_receptor4.py'),),
        ('-r', protein_pqr),
        ('-o', protein_pdbqt),
        ('-C',)]
).run()
retcode.check_returncode()

setting PYTHONHOME environment
